In [1]:
#import all the necessary packages

import pandas as pd
import random

import nltk
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk import stem
stemmer = stem.PorterStemmer()
from nltk import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
import string
punct = list(string.punctuation)
from collections import Counter
import requests
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
import numpy as np
import praw
import datetime
import json
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aoife\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aoife\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import PyPDF2

In [3]:
# open the PDF file
policy = open(r'C:\Users\aoife\Documents\Capstone\Capstone Quant Policies\Roehampton Disability.pdf', 'rb')

# create a PDF reader object
pdf_reader = PyPDF2.PdfReader(policy)

In [4]:
# read the text from each page
text = ''
for page_num in range(len(pdf_reader.pages)):
    page = pdf_reader.pages[page_num]
    text += page.extract_text()

# close the PDF file
policy.close()

In [5]:
# split the text into sentences
sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)

# create a dataframe with the sentences as rows
df = pd.DataFrame({'Sentence': sentences})

In [6]:
df

,Sentence
0,Disability Poli cy \nOwner: Director of St...
1,Introduction \n \n1.1 The University of Roeha...
2,\nThis policy outlines the University’s \ncomm...
3,The University \nanticipates the needs of staf...
4,\n \n1.2 The policy is guided by the Equality...
...,...
150,\n In order to maintain best practice in the ...
151,\n \nIn order to provide an appropriate level...
152,\n \nThis policy will be monitored and review...
153,This monitoring will be undertaken in \nconsul...


In [7]:
#check to see if any rows are empty
rows_with_nan = df[df.isna().any(axis=1)]

len(rows_with_nan)

0

In [8]:
#check the length of the dataframe
len(df)

155

In [9]:
# convert each row in the dataframe to a string and then convert the datadrame to a list of strings
p1 = df.apply(lambda row: ' '.join(row.astype(str)), axis=1).tolist()

In [10]:
p1

['   Disability Poli cy \nOwner:  Director of Student Support Services  \nReviewed by:  Student Life Committee  \nApproved by:  Senate (7July 2021 ) \nReview due date:  June 2025  \nUpdate history:  June 2012  \nTo discuss receiving the docum ent in an alternativ e format, please \ncontact the disabilities  team at disabilities@roehampton.ac.uk  1  DISABILITY POLICY  \n \n1.',
 'Introduction  \n \n1.1 The University of Roehampton is committed to promoting diversity and equality of \nopportunity and eliminating discrimination in its practices, policies and procedures, as set out in the University’s  Equality and Diversity P olicy.',
 '\nThis policy outlines the University’s \ncommitment to creating a culture and environment in which students, staff and visitors to the \nUniversity are treated fairly and not disadvantaged because of a disability.',
 'The University \nanticipates the needs of staff, students and visitors, and establishes inclusive practices, \npolicies and procedures  acc

In [11]:
# define a function to remove punctuation and numbers from a string
def remove_punctuation_and_numbers(text):
    # define a regular expression pattern to match punctuation characters and numbers
    pattern = r'[^A-Za-z\s]'
    # replace punctuation characters and numbers with an empty string
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

# apply the previously created function to each string in the list
cleaned_p1 = [remove_punctuation_and_numbers(string) for string in p1]

# display the cleaned list of strings
print(cleaned_p1)

['   Disability Poli cy \nOwner  Director of Student Support Services  \nReviewed by  Student Life Committee  \nApproved by  Senate July   \nReview due date  June   \nUpdate history  June   \nTo discuss receiving the docum ent in an alternativ e format please \ncontact the disabilities  team at disabilitiesroehamptonacuk    DISABILITY POLICY  \n \n', 'Introduction  \n \n The University of Roehampton is committed to promoting diversity and equality of \nopportunity and eliminating discrimination in its practices policies and procedures as set out in the Universitys  Equality and Diversity P olicy', '\nThis policy outlines the Universitys \ncommitment to creating a culture and environment in which students staff and visitors to the \nUniversity are treated fairly and not disadvantaged because of a disability', 'The University \nanticipates the needs of staff students and visitors and establishes inclusive practices \npolicies and procedures  accordingly', ' \n \n The policy is guided by 

In [12]:
# tokenize each string in the list of strings
tokenized_p1 = [word_tokenize(string) for string in cleaned_p1]

# display the tokenized list of strings
print(tokenized_p1)

[['Disability', 'Poli', 'cy', 'Owner', 'Director', 'of', 'Student', 'Support', 'Services', 'Reviewed', 'by', 'Student', 'Life', 'Committee', 'Approved', 'by', 'Senate', 'July', 'Review', 'due', 'date', 'June', 'Update', 'history', 'June', 'To', 'discuss', 'receiving', 'the', 'docum', 'ent', 'in', 'an', 'alternativ', 'e', 'format', 'please', 'contact', 'the', 'disabilities', 'team', 'at', 'disabilitiesroehamptonacuk', 'DISABILITY', 'POLICY'], ['Introduction', 'The', 'University', 'of', 'Roehampton', 'is', 'committed', 'to', 'promoting', 'diversity', 'and', 'equality', 'of', 'opportunity', 'and', 'eliminating', 'discrimination', 'in', 'its', 'practices', 'policies', 'and', 'procedures', 'as', 'set', 'out', 'in', 'the', 'Universitys', 'Equality', 'and', 'Diversity', 'P', 'olicy'], ['This', 'policy', 'outlines', 'the', 'Universitys', 'commitment', 'to', 'creating', 'a', 'culture', 'and', 'environment', 'in', 'which', 'students', 'staff', 'and', 'visitors', 'to', 'the', 'University', 'are',

In [13]:
# as some strings are now empty, remove empty strings from a list of strings
cleaned_list_of_strings = [string for string in tokenized_p1 if string]

# and just in case that doesn't work,, remove empty strings from a list of lists of strings
cleaned_tokenized_p1 = [[token for token in tokens if token] for tokens in tokenized_p1]

In [14]:
cleaned_tokenized_p1

[['Disability',
  'Poli',
  'cy',
  'Owner',
  'Director',
  'of',
  'Student',
  'Support',
  'Services',
  'Reviewed',
  'by',
  'Student',
  'Life',
  'Committee',
  'Approved',
  'by',
  'Senate',
  'July',
  'Review',
  'due',
  'date',
  'June',
  'Update',
  'history',
  'June',
  'To',
  'discuss',
  'receiving',
  'the',
  'docum',
  'ent',
  'in',
  'an',
  'alternativ',
  'e',
  'format',
  'please',
  'contact',
  'the',
  'disabilities',
  'team',
  'at',
  'disabilitiesroehamptonacuk',
  'DISABILITY',
  'POLICY'],
 ['Introduction',
  'The',
  'University',
  'of',
  'Roehampton',
  'is',
  'committed',
  'to',
  'promoting',
  'diversity',
  'and',
  'equality',
  'of',
  'opportunity',
  'and',
  'eliminating',
  'discrimination',
  'in',
  'its',
  'practices',
  'policies',
  'and',
  'procedures',
  'as',
  'set',
  'out',
  'in',
  'the',
  'Universitys',
  'Equality',
  'and',
  'Diversity',
  'P',
  'olicy'],
 ['This',
  'policy',
  'outlines',
  'the',
  'Universi

In [15]:
#lemmatize
postlemma = []

for i in cleaned_tokenized_p1: 
    postlemma.append([lemmatizer.lemmatize(j) for j in i])

In [16]:
postlemma

[['Disability',
  'Poli',
  'cy',
  'Owner',
  'Director',
  'of',
  'Student',
  'Support',
  'Services',
  'Reviewed',
  'by',
  'Student',
  'Life',
  'Committee',
  'Approved',
  'by',
  'Senate',
  'July',
  'Review',
  'due',
  'date',
  'June',
  'Update',
  'history',
  'June',
  'To',
  'discus',
  'receiving',
  'the',
  'docum',
  'ent',
  'in',
  'an',
  'alternativ',
  'e',
  'format',
  'please',
  'contact',
  'the',
  'disability',
  'team',
  'at',
  'disabilitiesroehamptonacuk',
  'DISABILITY',
  'POLICY'],
 ['Introduction',
  'The',
  'University',
  'of',
  'Roehampton',
  'is',
  'committed',
  'to',
  'promoting',
  'diversity',
  'and',
  'equality',
  'of',
  'opportunity',
  'and',
  'eliminating',
  'discrimination',
  'in',
  'it',
  'practice',
  'policy',
  'and',
  'procedure',
  'a',
  'set',
  'out',
  'in',
  'the',
  'Universitys',
  'Equality',
  'and',
  'Diversity',
  'P',
  'olicy'],
 ['This',
  'policy',
  'outline',
  'the',
  'Universitys',
  'c

In [17]:
#lower all cases
postfin1 = [[j.lower() for j in i] for i in postlemma]

In [18]:
postfin1

[['disability',
  'poli',
  'cy',
  'owner',
  'director',
  'of',
  'student',
  'support',
  'services',
  'reviewed',
  'by',
  'student',
  'life',
  'committee',
  'approved',
  'by',
  'senate',
  'july',
  'review',
  'due',
  'date',
  'june',
  'update',
  'history',
  'june',
  'to',
  'discus',
  'receiving',
  'the',
  'docum',
  'ent',
  'in',
  'an',
  'alternativ',
  'e',
  'format',
  'please',
  'contact',
  'the',
  'disability',
  'team',
  'at',
  'disabilitiesroehamptonacuk',
  'disability',
  'policy'],
 ['introduction',
  'the',
  'university',
  'of',
  'roehampton',
  'is',
  'committed',
  'to',
  'promoting',
  'diversity',
  'and',
  'equality',
  'of',
  'opportunity',
  'and',
  'eliminating',
  'discrimination',
  'in',
  'it',
  'practice',
  'policy',
  'and',
  'procedure',
  'a',
  'set',
  'out',
  'in',
  'the',
  'universitys',
  'equality',
  'and',
  'diversity',
  'p',
  'olicy'],
 ['this',
  'policy',
  'outline',
  'the',
  'universitys',
  'c

In [19]:
# replace the old comments with the new pre-processed comments
data = {'Sentence': postfin1}
p1 = pd.DataFrame(data)

In [20]:
p1

,Sentence
0,"[disability, poli, cy, owner, director, of, st..."
1,"[introduction, the, university, of, roehampton..."
2,"[this, policy, outline, the, universitys, comm..."
3,"[the, university, anticipates, the, need, of, ..."
4,"[the, policy, is, guided, by, the, equality, a..."
...,...
150,"[in, order, to, maintain, best, practice, in, ..."
151,"[in, order, to, provide, an, appropriate, leve..."
152,"[this, policy, will, be, monitored, and, revie..."
153,"[this, monitoring, will, be, undertaken, in, c..."


In [21]:
# set display options to show all columns and rows in the dataframe so i can quickly scan through
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

# display the dataframe
print(p1)

                                              Sentence
0    [disability, poli, cy, owner, director, of, st...
1    [introduction, the, university, of, roehampton...
2    [this, policy, outline, the, universitys, comm...
3    [the, university, anticipates, the, need, of, ...
4    [the, policy, is, guided, by, the, equality, a...
5                                                   []
6    [scope, this, policy, cover, the, universitys,...
7                                                   []
8    [responsibilities, the, university, s, equalit...
9    [members, of, the, universitys, senior, admini...
10   [all, staff, should, be, aware, of, their, sha...
11   [advice, support, and, training, will, be, pro...
12   [human, resources, will, ensure, that, all, st...
13   [all, staff, student, and, visitor, to, the, u...
14                                                  []
15   [definitions, equality, act, the, equality, ac...
16   [this, policy, cover, the, protected, characte...
17   [unde

In [22]:
# Doing so helped me notice that there are a bunch of empty lists, so we will remove those now
# find rows with empty lists in the 'Sentence' column
rows_with_empty_lists = p1[p1['Sentence'].apply(len) == 0]

# display rows with empty lists
print(rows_with_empty_lists)

    Sentence
5         []
7         []
14        []
26        []
130       []
144       []


In [23]:
# remove rows with empty lists in the 'Sentence' column
p1 = p1[p1['Sentence'].apply(len) > 0]

# reset index
p1.reset_index(drop=True, inplace=True)

# display the modified DataFrame
print(p1)

                                              Sentence
0    [disability, poli, cy, owner, director, of, st...
1    [introduction, the, university, of, roehampton...
2    [this, policy, outline, the, universitys, comm...
3    [the, university, anticipates, the, need, of, ...
4    [the, policy, is, guided, by, the, equality, a...
5    [scope, this, policy, cover, the, universitys,...
6    [responsibilities, the, university, s, equalit...
7    [members, of, the, universitys, senior, admini...
8    [all, staff, should, be, aware, of, their, sha...
9    [advice, support, and, training, will, be, pro...
10   [human, resources, will, ensure, that, all, st...
11   [all, staff, student, and, visitor, to, the, u...
12   [definitions, equality, act, the, equality, ac...
13   [this, policy, cover, the, protected, characte...
14   [under, the, equality, act, the, university, h...
15   [disability, disability, is, defined, a, a, ph...
16   [day, today, activity, are, not, defined, in, ...
17   [peop

In [24]:
# just to check it is still all good in this format too (can't be too careful)
p1

,Sentence
0,"[disability, poli, cy, owner, director, of, st..."
1,"[introduction, the, university, of, roehampton..."
2,"[this, policy, outline, the, universitys, comm..."
3,"[the, university, anticipates, the, need, of, ..."
4,"[the, policy, is, guided, by, the, equality, a..."
5,"[scope, this, policy, cover, the, universitys,..."
6,"[responsibilities, the, university, s, equalit..."
7,"[members, of, the, universitys, senior, admini..."
8,"[all, staff, should, be, aware, of, their, sha..."
9,"[advice, support, and, training, will, be, pro..."


In [25]:
# save the dataset to be able to use in analysis

#export to excel file
p1.to_excel('p6.xlsx')

#export to csv file
p1.to_csv('p6.csv', index=False)